In [13]:
from re import M
import torch
import torch.optim as optim
import os
import torch.nn as nn
from yolo import Yolo
from loss import Loss
from utils import  AverageMeter,class_accuracy
from dataset import get_data
import warnings
import time
import numpy as np
warnings.filterwarnings("ignore")

In [4]:
model = Yolo(3,6//2,2)
model.load_state_dict(torch.load('model_1111.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [30]:
x = torch.rand(1,3,416,416)
with torch.no_grad():
    out_feat = model(x)
print(out_feat[0].shape, out_feat[1].shape)
out13 = torch.reshape(out_feat[0], (1,21,13,13))
out26 = torch.reshape(out_feat[1], (1,21,26,26))
print(out13.shape, out26.shape)
my_out = [out13, out26]

torch.Size([1, 3, 13, 13, 7]) torch.Size([1, 3, 26, 26, 7])
torch.Size([1, 21, 13, 13]) torch.Size([1, 21, 26, 26])


In [69]:
class DecodeBox(nn.Module):
    def __init__(self, scaled_anchors, num_classes, img_size):
        super(DecodeBox, self).__init__()
        self.scaled_anchors = scaled_anchors
        self.num_anchors = len(scaled_anchors)
        self.num_classes = num_classes
        self.bbox_attrs = 5 + num_classes
        self.img_size = img_size

    def forward(self, input):
        batch_size = input.size(0)
        input_height = input.size(2)
        input_width = input.size(3)
        stride_h = self.img_size[1] / input_height
        stride_w = self.img_size[0] / input_width
        scaled_anchors = self.scaled_anchors

        prediction = input.view(batch_size, self.num_anchors,
                                self.bbox_attrs, input_height, input_width).permute(0, 1, 3, 4, 2).contiguous()
        x = torch.sigmoid(prediction[..., 0])
        y = torch.sigmoid(prediction[..., 1])
        w = prediction[..., 2]
        h = prediction[..., 3]
        conf = torch.sigmoid(prediction[..., 4])
        pred_cls = torch.sigmoid(prediction[..., 5:])
        FloatTensor = torch.cuda.FloatTensor if x.is_cuda else torch.FloatTensor
        LongTensor = torch.cuda.LongTensor if x.is_cuda else torch.LongTensor
        grid_x = torch.linspace(0, input_width - 1, input_width).repeat(input_height, 1).repeat(
            batch_size * self.num_anchors, 1, 1).view(x.shape).type(FloatTensor)
        grid_y = torch.linspace(0, input_height - 1, input_height).repeat(input_width, 1).t().repeat(
            batch_size * self.num_anchors, 1, 1).view(y.shape).type(FloatTensor)
        anchor_w = FloatTensor(scaled_anchors).index_select(1, LongTensor([0]))
        anchor_h = FloatTensor(scaled_anchors).index_select(1, LongTensor([1]))
        anchor_w = anchor_w.repeat(batch_size, 1).repeat(1, 1, input_height * input_width).view(w.shape)
        anchor_h = anchor_h.repeat(batch_size, 1).repeat(1, 1, input_height * input_width).view(h.shape)
        pred_boxes = FloatTensor(prediction[..., :4].shape)
        pred_boxes[..., 0] = x.data + grid_x
        pred_boxes[..., 1] = y.data + grid_y
        pred_boxes[..., 2] = torch.exp(w.data) * anchor_w
        pred_boxes[..., 3] = torch.exp(h.data) * anchor_h
        scale = torch.Tensor([stride_w, stride_h] * 2).type(FloatTensor)
        output = torch.cat((pred_boxes.view(batch_size, -1, 4) * scale,
                            conf.view(batch_size, -1, 1), pred_cls.view(batch_size, -1, self.num_classes)), -1)
        return output.data

In [70]:
S = [13,26]
ANCHORS =  [[(0.275 ,   0.320312), (0.068   , 0.113281), (0.017  ,  0.03   )], 
           [(0.03  ,   0.056   ), (0.01  ,   0.018   ), (0.006 ,   0.01    )]]

scaled_anchors = (
        torch.tensor(ANCHORS)
        * torch.tensor(S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
    ).to("cpu")

In [71]:
feat_decoder=[]
anchor_arr=[[3,4,5],[0,1,2]]
for i in range(2):
    decoder=DecodeBox(np.reshape(scaled_anchors,[-1,2])[anchor_arr[i]],2,(416,416))
    feat_decoder.append(decoder)
    
out_list=[]
for i in range(2):
        decoder_out=feat_decoder[i](my_out[i])
        out_list.append(decoder_out)
                
output=torch.cat(out_list,dim=1)

In [74]:
output.shape

torch.Size([1, 2535, 7])

In [75]:
box_temp=output.new(output.shape)
box_temp[:,:,0]=output[:,:,0]-output[:,:,2]/2
box_temp[:,:,1]=output[:,:,1]-output[:,:,3]/2
box_temp[:,:,2]=output[:,:,0]+output[:,:,2]/2
box_temp[:,:,3]=output[:,:,1]+output[:,:,3]/2
output[:,:,:4]=box_temp[:,:,:4]

In [85]:
for idx, p in enumerate(output):
    cls_conf,cls_pred=torch.max(p[:,5:7],dim=1,keepdim=True)
    score = p[:,4]*cls_conf[:,0]
    

In [86]:
cls_conf, cls_pred, score

(tensor([[0.4612],
         [0.6822],
         [0.6656],
         ...,
         [0.7613],
         [0.4670],
         [0.8994]]),
 tensor([[1],
         [0],
         [1],
         ...,
         [1],
         [0],
         [1]]),
 tensor([0.2506, 0.4196, 0.2725,  ..., 0.4478, 0.3850, 0.0675]))

In [84]:
(score > 0.75

tensor([False, False, False,  ..., False, False, False])

In [79]:
len(cls_conf)

2535

In [73]:
output[:,1,:]

tensor([[5.1037e+01, 2.0090e+01, 2.3804e+01, 5.8630e+01, 6.1508e-01, 6.8219e-01,
         5.4797e-05]])